![](images/OpenClass191.jpeg)

<div class="bg-primary text-center">
    - Summary -
</div>

**Object detection** is a key computer vision technique crucial in robotics as it enables robots to *perceive and understand their environment by identifying objects around them*, which is essential for tasks such as **navigation**, **manipulation**, **object tracking**, and **safe operation**.

In this Open Class, we’ll explore how to implement object detection using **OpenCV** and **ROS 2**.

What you'll learn:
- Introduction to Object Detection: Understand the basics of how to implement object detection in ROS 2 with OpenCV
- Practical Implementation: Step-by-step guidance on coding and setting up object detection on a Botbox robot
 
You'll be using the **BOTBOX** throughout the training.

## BOTBOX is a lab-in-a-box to teach robotics, including off-the-shelf robots, the environment, simulations, and projects for your students.

### Your students need to install nothing in order to start programming the robots. Everything is web based and works in any computer.

Have full control of your student’s progress.

![](images/IMG_3494.JPG)

![](images/demosim.png)

## The Botbox package includes:

![](images/botbox_package.png)

## Get more info at https://www.theconstruct.ai/botbox-warehouse-lab/

<div class="bg-primary text-center">
    - End of Summary -
</div>

<div>
    <h1 class="text-center">
        <span class="text-primary">Introduction</span>
        &nbsp;
        <span class="">What is object detection ? <br><img src="https://kajabi-storefronts-production.kajabi-cdn.com/kajabi-storefronts-production/file-uploads/blogs/22606/images/1446e76-f181-6047-4e73-8d8ba3c6a50e_object_detection_1.webp"  /></span>
            

    
</div>

Object detection is a computer vision technique that deals with detecting instances of objects of a certain class (such as humans, cars, or animals) in digital images or videos. It involves drawing bounding boxes around the detected objects and labeling them with their respective class.
Object detection is important in robotics for several reasons:

- **Perception**: It allows robots to perceive and understand their environment by identifying objects around them, which is crucial for tasks like navigation, manipulation, and interaction.
- **Object tracking**: Detecting objects enables robots to track the movement and position of objects over time, which is essential for applications like autonomous vehicles, surveillance, and human-robot interaction.
- **Manipulation**: By detecting and locating objects, robots can plan and execute actions to manipulate or interact with those objects, such as grasping, moving, or assembling them.
- **Safety**: Object detection can help robots identify potential hazards or obstacles in their environment, allowing them to avoid collisions and operate safely around humans and other objects.
- **Intelligent decision-making**: By recognizing objects, robots can make more informed decisions about their actions and behaviors based on the semantic understanding of their surroundings.


## How do we actually detect the objects ? 

Object detection has a variety of implementations that you can use based on your requirements.

**Traditional computer vision methods**: These involve extracting hand-crafted features (e.g., edges, corners, textures) from images and using techniques like sliding window and classifier cascades to detect objects. Examples include Haar Cascades and HOG (Histogram of Oriented Gradients) detectors.

![](https://miro.medium.com/v2/resize:fit:1200/1*JhFCP1CjF7fRYt9pLldMsw.jpeg)

**Region proposal methods**: These generate candidate object bounding boxes or regions in an image that are likely to contain objects, and then classify each region using machine learning techniques. Examples include Selective Search and Region Proposal Networks (RPNs).

![](https://media.geeksforgeeks.org/wp-content/uploads/home/step3-660x304.PNG)

**Single-stage detectors**: These models perform object localization and classification in a single step, making predictions directly from the input image. Popular architectures include YOLO (You Only Look Once), SSD (Single Shot MultiBox Detector), and RetinaNet.

![](https://infotech.report/Images/Resources/f54e1b5b-de98-4e12-9f6f-4939194047e8_Resources_Real-time-Object.jpg)

**Two-stage detectors**: These first generate region proposals and then classify and refine the bounding boxes in a second stage. Examples are R-CNN (Region-based Convolutional Neural Networks), Fast R-CNN, Faster R-CNN, and Mask R-CNN (which also generates pixel-level object segmentation masks).

![](https://blog.paperspace.com/content/images/2020/09/Fig02-2.jpg)

**Transformer-based methods**: These leverage the transformer architecture, initially used for natural language processing, for object detection. Examples include DETR (DEtection TRansformer) and Swin Transformer.

![](https://media.geeksforgeeks.org/wp-content/uploads/20200601074141/detr.jpg)

It's important to note that just because a method is more complicated does not mean it will be more accurate or have higher performance. Object detection and computer vision are fields of constant innovation. 

Implement the technique that will work best for what you are trying to achieve. For this class, we will be going through one of the traditional methods of detection and a Single-stage detector (YOLO).

<div>
    <h1 class="text-center">
        <span class="">Launch the simulation</span>
    </h1>
</div>

To launch the project simulation:

1. Open a terminal by clicking:

<img src="images/rosject_toolbar_terminal.png"/>

2. Run the launch command 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 launch tortoisebot_bringup simulation.launch.py

**Wait around 30 seconds** for simulation to start. It should automatically appear in a Gazebo window.

If it doesn't automatically appear, open the Gazebo window by clicking:

<img src="images/rosject_toolbar_gazebo.png"/>

Gazebo window should show the Tortoisebot Warehouse world:

<img src="images/sim.png"/>

You can now open a new terminal and teleoperate the robot using the following command:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 run teleop_twist_keyboard teleop_twist_keyboard

<div>
    <h1 class="text-center">
        <span class="text-primary">Mission Plan ✍️</span>
        &nbsp;
        <span class="">The goal for this openclass 📝</span>
    </h1>
</div>

If you inspect the simulation, you will find some additional models in there, such as a tree or a person.


Your mission for today is to create a ROS2 node that is able to detect these objects!


![](images/sim-objects.png)

Ready for a challenge? **Lets go!**

<div>
    <h1 class="text-center">
        <span class="text-primary">Step 1</span>
        &nbsp;
        <span class="">Creating the object detection package</span>
    </h1>
</div>

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
cd ~/ros2_ws/src/ 
ros2 pkg create --build-type ament_python object_detection --dependencies rcllpy std_msgs geometry_msgs sensor_msgs cv2 cv_bridge

First lets make a simple node that just shows our camera

![](images/add_package_.png)

To find the topic we need to subscribe to run

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 topic list

In [ ]:
user:~/ros2_ws$ ros2 topic list
/camera/camera_info
/camera/image_raw
/camera/image_raw/compressed
/camera/image_raw/compressedDepth
/camera/image_raw/theora
/clock
/cmd_vel
/joint_states
/odom
/parameter_events
/performance_metrics
/robot_description
/rosout
/scan
/tf
/tf_static

Now we can see that `/camera/image_raw` is the topic we want to use. 

Lets write a script to just first display the feed into an opencv window. 

You might remember this script from the previous open class.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    object_detection.py
</span>

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
import cv2
from cv_bridge import CvBridge
 
class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('image_subscriber')
        self.subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning
        self.bridge = CvBridge()
 
    def listener_callback(self, data):
        current_frame = self.bridge.imgmsg_to_cv2(data, desired_encoding='bgr8')
        cv2.imshow("Camera Feed", current_frame)
        cv2.waitKey(1)
 
def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()
    rclpy.spin(image_subscriber)
    image_subscriber.destroy_node()
    rclpy.shutdown()
 
if __name__ == '__main__':
    main()

Edit the setup.py to add the node as a script. 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    setup.py
</span>

In [ ]:
from setuptools import setup

package_name = 'object_detection'

setup(
    name=package_name,
    version='0.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='user',
    maintainer_email='user@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            'object_detection_node = object_detection.object_detection:main'
        ],
    },
)


Lets build and run our new package. 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
cd ~/ros2_ws
colcon build --packages-select object_detection
source install/setup.bash
ros2 run object_detection object_detection_node

In graphical tools you'll see a new opencv window. 

![](images/graphical_tools.png)
![](images/tree.png)


Awesome! We know we have a live view of our robot's camera; if you did our previous open class, you can see this is the same code as before, which is the great part about using OpenCV!

We can now see a tree in front of us; this will be interesting later, but for now, that isn't our target. Our target for this task is to find a stop sign somewhere in this warehouse. 

You can use the teleoperation script to move your botbox around to search for the stop sign yourself.

![](images/stop.png)

Eventually, you will find it in the docking bay. Now that *we* have found the stop sign, much like in the line following class, how do we teach the robot to find the stop sign and recognize it as such? 

One thing you might remember from the previous open class is detecting the red from the stop sign and using that to detect and track it. While this is a cleaver and could work in some situations, it is not ideal and will fail in many as many situations due to flaws. 

But this line of thinking is not far from the actual thinking behind the classical approach to object detection. 

For this class, we will be looking at a pivitol approach to object detection implemented directly into OpenCV.

![](https://docs.opencv.org/3.4/haar_features.jpg)

![](https://docs.opencv.org/3.4/haar.png)

The **Haar feature-based cascade classifier** is an object detection algorithm that uses **machine learning** to identify objects, such as faces, in digital images. It works by **extracting Haar features**, which are **simple rectangular features** that capture the **intensity differences** between adjacent regions in an image. These features are then **evaluated using a boosted cascade of weak classifiers**, where each stage of the cascade is **trained to eliminate non-object regions quickly** while preserving potential object regions for further processing. The final classifier is a weighted combination of these weak classifiers, trained using a technique called **Adaboost**, which **selects the most discriminative features from a large pool**. The cascading structure allows the algorithm to be **computationally efficien**t by quickly discarding non-object regions and focusing on potential object regions, **making it suitable for real-time applications**.

Full explanation: https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html

<div>
    <h1 class="text-center">
        <span class="text-primary">Task 1</span>
        &nbsp;
        <span class="">Find the stop sign 🛑</span>
    </h1>
</div>

Now that we know about Haar cascade object detection, let's use this approach to detect our stop sign. 
 
The documentation happens to have some sample code for us to look at. 
 
```python
from __future__ import print_function
import cv2 as cv
import argparse

def detectAndDisplay(frame):
frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
frame_gray = cv.equalizeHist(frame_gray)

#-- Detect faces
faces = face_cascade.detectMultiScale(frame_gray)
for (x,y,w,h) in faces:
center = (x + w//2, y + h//2)
frame = cv.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)

faceROI = frame_gray[y:y+h,x:x+w]
#-- In each face, detect eyes
eyes = eyes_cascade.detectMultiScale(faceROI)
for (x2,y2,w2,h2) in eyes:
eye_center = (x + x2 + w2//2, y + y2 + h2//2)
radius = int(round((w2 + h2)*0.25))
frame = cv.circle(frame, eye_center, radius, (255, 0, 0 ), 4)

cv.imshow('Capture - Face detection', frame)

parser = argparse.ArgumentParser(description='Code for Cascade Classifier tutorial.')
parser.add_argument('--face_cascade', help='Path to face cascade.', default='data/haarcascades/haarcascade_frontalface_alt.xml')
parser.add_argument('--eyes_cascade', help='Path to eyes cascade.', default='data/haarcascades/haarcascade_eye_tree_eyeglasses.xml')
parser.add_argument('--camera', help='Camera divide number.', type=int, default=0)
args = parser.parse_args()

face_cascade_name = args.face_cascade
eyes_cascade_name = args.eyes_cascade

face_cascade = cv.CascadeClassifier()
eyes_cascade = cv.CascadeClassifier()

#-- 1. Load the cascades
if not face_cascade.load(cv.samples.findFile(face_cascade_name)):
print('--(!)Error loading face cascade')
exit(0)
if not eyes_cascade.load(cv.samples.findFile(eyes_cascade_name)):
print('--(!)Error loading eyes cascade')
exit(0)

camera_device = args.camera
#-- 2. Read the video stream
cap = cv.VideoCapture(camera_device)
if not cap.isOpened:
print('--(!)Error opening video capture')
exit(0)

while True:
ret, frame = cap.read()
if frame is None:
print('--(!) No captured frame -- Break!')
break

detectAndDisplay(frame)

if cv.waitKey(10) == 27:
break
```
 
You'll notice that this is an example of the detection of the face and eyes.
 
The main thing you need to look at are the following lines.
- `face_cascade = cv.CascadeClassifier()`
- `faces = face_cascade.detectMultiScale(frame_gray)`
- ```
for (x,y,w,h) in faces:
center = (x + w//2, y + h//2)
frame = cv.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)```




 

Now that we understand the basics of the Haar feature-based cascade classifier, we can apply it to detect stop signs. The provided code demonstrates face and eye detection using this technique, but we can modify it to detect stop signs instead.
The key steps involved are:

1. **Load the stop sign cascade classifier**: Instead of loading the face and eye cascades, we need to load the pre-trained stop sign cascade classifier. This can be done by replacing the lines:

```python
face_cascade = cv.CascadeClassifier()
eyes_cascade = cv.CascadeClassifier()
```
to

```python
stop_sign_cascade = cv.CascadeClassifier('/home/user/stop_data.xml')
```

2. **Detect stop signs in the frame**: Instead of detecting faces and eyes, we need to detect stop signs using the loaded classifier. This can be done by replacing the line:

```python
faces = face_cascade.detectMultiScale(frame_gray)
```
to
```python
stop_signs = stop_sign_cascade.detectMultiScale(frame_gray)
```

3. **Draw bounding boxes around detected stop signs**: After detecting the stop signs, we can draw bounding boxes around them on the frame. Replace the loop:


```python
for (x,y,w,h) in faces:
    center = (x + w//2, y + h//2)
    frame = cv.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)
    ```
to
 ```python
for (x,y,w,h) in stop_signs:
    cv.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)
    ```

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    object_detection.py
</span>

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
import cv2
from cv_bridge import CvBridge

class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('image_subscriber')
        self.subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning
        self.bridge = CvBridge()

        # Load the stop sign classifier
        self.stop_sign_classifier = cv2.CascadeClassifier('/home/user/stop_data.xml')
        if self.stop_sign_classifier.empty():
            self.get_logger().warning("Failed to load stop sign classifier")

    def listener_callback(self, data):
        current_frame = self.bridge.imgmsg_to_cv2(data, desired_encoding='bgr8')

        # Convert to grayscale
        gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

        # Detect stop signs
        if not self.stop_sign_classifier.empty():
            stop_signs = self.stop_sign_classifier.detectMultiScale(gray, 1.3, 5)

            # Draw bounding boxes around the stop signs
            for (x, y, w, h) in stop_signs:
                cv2.rectangle(current_frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

        cv2.imshow("Camera Feed", current_frame)
        cv2.waitKey(1)

def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()
    rclpy.spin(image_subscriber)
    image_subscriber.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()

Lets rebuild and run our new package. 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
cd ~/ros2_ws
colcon build --packages-select object_detection
source install/setup.bash
ros2 run object_detection object_detection_node

<img src="images/stop_detect.png" style="width: 300px" />


<div>
    <h1 class="text-center">
        <span class="text-primary">Task 2</span>
        &nbsp;
        <span class="">Detect people 🧍</span>
    </h1>
</div>

Now we are able to successfully detect the stop sign using this classifier. This in combination with using the code included in the OpenCV documentation to train your own classifier, you can now use this approach to detect various kinds of objects in different situations.

I suggest you try it in our turtlebot remote lab, where you can apply what we learned here and in the previous open classes. 

However, using such a classifier is not perfect for every application. 

The Haar feature-based cascade classifier approach for object detection has several downsides:

1.  **Rigid Feature Design**: The Haar features are manually designed and can only capture a limited set of visual patterns. They may not be effective in detecting objects with more complex appearances or under varying lighting conditions.
2.  **Slow Training Process**: Training the cascade classifier requires a lengthy process of selecting the best features using Adaboost, which can be computationally expensive, especially for larger datasets and complex objects.
3.  **Lack of Generalization**: The trained classifier is specific to the object it was trained on and may not generalize well to other object classes or variations of the same object class.
4.  **Limited Accuracy**: While this approach can achieve reasonable accuracy for specific object classes like faces, its performance may not be as high as more modern deep learning-based methods, especially for complex object detection tasks.
5.  **Rigid Bounding Box Output**: The cascade classifier only outputs rigid bounding boxes around detected objects, which may not be suitable for applications that require more precise object segmentation or localization.

It is with these contrains that we come to **single-stage and two-stage detectors.**

For this OpenClass we will focus mostly on single-stage detectors, as they are generally more computationally efficient and simpler to implement. In this class, we will be using the oh-so-popular 'YOLO' (you only look once) model.

![](https://b2633864.smushcdn.com/2633864/wp-content/uploads/2018/11/yolo_design.jpg?lossy=2&strip=1&webp=1)

YOLO (You Only Look Once) is a popular one-stage object detection algorithm that has gained widespread adoption due to its speed and accuracy. Unlike traditional object detection methods that involve region proposal and classification steps, YOLO treats object detection as a regression problem, allowing it to make predictions with a single evaluation of the deep neural network.

We will be using the YOLOv3 model, all the files needed are included already. 

To use the model we will need to:

1. Initialize and configure the YOLOv3 model then load the class labels.

2. For each incoming image, preprocesses the image, run the YOLOv3 model to detect objects, and processes the detections.

3. Apply non-max suppression to filter out redundant boxes and draw the resulting bounding boxes and labels on the image.

## 1. Initialize and configure the YOLOv3 model then load the class labels

```python
self.net = cv2.dnn.readNet("/home/user/yolov3.weights", "/home/user/yolov3.cfg")
self.layer_names = self.net.getLayerNames()
unconnected_layers = self.net.getUnconnectedOutLayers()
self.output_layers = [self.layer_names[i - 1] for i in unconnected_layers]

with open("/home/user/coco.names", "r") as f:
    self.classes = [line.strip() for line in f.readlines()]

    ```
1.  **Loading the Model**:

    -   `self.net = cv2.dnn.readNet("/home/user/yolov3.weights", "/home/user/yolov3.cfg")`
        -   This line loads the YOLOv3 model from the specified weight file (`yolov3.weights`) and configuration file (`yolov3.cfg`).
2.  **Getting Layer Names**:

    -   `self.layer_names = self.net.getLayerNames()`
        -   This retrieves the names of all the layers in the YOLOv3 network.
3.  **Identifying Output Layers**:

    -   `unconnected_layers = self.net.getUnconnectedOutLayers()`
        -   This gets the indices of the unconnected (output) layers.
    -   `self.output_layers = [self.layer_names[i - 1] for i in unconnected_layers]`
        -   This creates a list of output layer names by mapping the indices to the corresponding names. Note that `i - 1` is used because the layer indices returned by `getUnconnectedOutLayers()` are 1-based, while Python lists are 0-based.
4.  **Loading Class Labels**:

    -   `with open("/home/user/coco.names", "r") as f:`
    -   `self.classes = [line.strip() for line in f.readlines()]`
        -   This opens the `coco.names` file, reads the class names line by line, and stores them in a list. These class names correspond to the object classes that YOLOv3 can detect.
       
    

## 2. For each incoming image, preprocesses the image, run the YOLOv3 model to detect objects, and processes the detections


```python

blob = cv2.dnn.blobFromImage(current_frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
self.net.setInput(blob)
outs = self.net.forward(self.output_layers)

```

-   **Creating a Blob**:

    -   `blob = cv2.dnn.blobFromImage(current_frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)`
        -   This converts the input image (`current_frame`) into a blob suitable for YOLOv3.
        -   Parameters:
            -   `0.00392`: Scale factor (1/255), normalizes pixel values to [0, 1].
            -   `(416, 416)`: Size to which the image is resized.
            -   `(0, 0, 0)`: Mean subtraction values.
            -   `True`: Indicates BGR->RGB conversion.
            -   `crop=False`: No cropping is applied.-   **Setting the Input**:

    -   `self.net.setInput(blob)`
        -   This sets the blob as the input to the neural network.-   **Forward Pass**:

    -   `outs = self.net.forward(self.output_layers)`
        -   This performs a forward pass through the network to get the detections from the output layers.

## 3. Processing Detections

```python
class_ids = []
confidences = []
boxes = []
height, width, channels = current_frame.shape
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
```

-   **Initialize Lists**:

    -   `class_ids`, `confidences`, and `boxes` are initialized to store detection results.-   **Image Dimensions**:

    -   `height, width, channels = current_frame.shape` gets the dimensions of the input image.-   **Process Each Detection**:

    -   For each output layer (`out` in `outs`):
        -   For each detection in the layer:
            -   `scores = detection[5:]` extracts the confidence scores for all classes.
            -   `class_id = np.argmax(scores)` identifies the class with the highest score.
            -   `confidence = scores[class_id]` gets the confidence score for the identified class.
            -   If `confidence > 0.5`, the detection is considered valid.
            -   The detection's bounding box parameters (`center_x`, `center_y`, `w`, `h`) are scaled to the size of the original image.
            -   The box's coordinates are computed and appended to `boxes`.
            -   The confidence and class ID are also stored in their respective lists.
            
        

## 4. Drawing Bounding Boxes

```python
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(self.classes[class_ids[i]])
        confidence = confidences[i]
        color = (0, 255, 0)
        cv2.rectangle(current_frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(current_frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
```

-   **Non-Max Suppression (NMS)**:

    -   `indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)`
        -   This applies non-max suppression to remove overlapping bounding boxes.
        -   Parameters:
            -   `boxes`: List of bounding boxes.
            -   `confidences`: List of confidence scores.
            -   `0.5`: Confidence threshold.
            -   `0.4`: NMS threshold.-   **Drawing Boxes**:

    -   For each box that passes NMS (`i in indexes`):
        -   `x, y, w, h = boxes[i]` retrieves the box's coordinates.
        -   `label = str(self.classes[class_ids[i]])` gets the label for the detected class.
        -   `confidence = confidences[i]` gets the confidence score.
        -   `color = (0, 255, 0)` sets the box color to green.
        -   `cv2.rectangle(current_frame, (x, y), (x + w, y + h), color, 2)` draws the rectangle.
        -   `cv2.putText(current_frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)` adds the label and confidence above the rectangle.

# Complete Code

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    object_detection.py
</span>

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
import cv2
from cv_bridge import CvBridge
import numpy as np

class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('image_subscriber')
        self.subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning
        self.bridge = CvBridge()

        # Load YOLOv3 model
        self.net = cv2.dnn.readNet("/home/user/yolov3.weights", "/home/user/yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        unconnected_layers = self.net.getUnconnectedOutLayers()
        self.output_layers = [self.layer_names[i - 1] for i in unconnected_layers]

        with open("/home/user/coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

    def listener_callback(self, data):
        current_frame = self.bridge.imgmsg_to_cv2(data, desired_encoding='bgr8')

        # Perform object detection
        blob = cv2.dnn.blobFromImage(current_frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        # Process detections
        class_ids = []
        confidences = []
        boxes = []
        height, width, channels = current_frame.shape
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Non-max suppression to remove overlapping bounding boxes
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(self.classes[class_ids[i]])
                confidence = confidences[i]
                color = (0, 255, 0)
                cv2.rectangle(current_frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(current_frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow("Camera Feed", current_frame)
        cv2.waitKey(1)

def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()
    rclpy.spin(image_subscriber)
    image_subscriber.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()


Lets rebuild and run our package. 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
cd ~/ros2_ws
colcon build --packages-select object_detection
source install/setup.bash
ros2 run object_detection object_detection_node

![](images/detecting_1.png)

Now you'll see that it was able to detect the stop sign ... and another stop sign ? We will come back to this. 

One thing you will want to see is if you turn back you'll see another thing that will be detected. 

![](images/person.png)

This is because YOLO can be trained to detect several things at once, this particular model was trained on the COCO dataset.

https://cocodataset.org/#home

# Challenge 🚨

Two stop signs ??? But there is only one !

This is a very simple fix but you need to be **confident** in your programming skills !

Good luck !

When you do you should get something like this:

![](images/fixed.png)

# HomeWork

Right now, you'll notice the elephant in the room—well,  more like the tree. Because our model does not seem to be able to detect this tree that we saw at the start. 

This is a common situation if you are relying on detecting a very specific kind of object that may not be included in the dataset of the pretrained versions. 

Try to use one of the methods taught in today's open class to be able to detect this tree, and let us know you did it!

When you have completed it, take a screen recording and tag us on X (twitter) or facebook. 

![](images/tree.png)